<p style="padding: 10px; background-color:#FFA500;color: Black;font-weight: bold;
              text-align: center; font-size:220%;border-radius:10px 80px;">Обучение пайплайна</p>

Используем набор данных https://www.kaggle.com/datasets/pritsheta/heart-attack?select=Heart+Attack+Data+Set.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Heart Attack Data Set.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [5]:
df.nunique()

age          41
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64

In [6]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

    
from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
cat_cols = ['sex','cp', 'fbs', 'exang', 'slope']
base_cols = ['restecg', 'ca', 'thal']


continuos_transformers = []
cat_transformers = []
base_transformers=[]


for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [7]:
from sklearn.model_selection import train_test_split
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1), 
                                                    df['target'], random_state=24)

/tmp/ipykernel_10821/4128685706.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train, X_test, y_train, y_test = train_test_split(df.drop('target', 1),


In [8]:
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [9]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-6.75941412e-01, -4.34565489e-01,  5.14025007e-01, ...,
         0.00000000e+00,  0.00000000e+00,  3.00000000e+00],
       [ 5.49785317e-01,  1.37629209e-01, -8.13865964e-01, ...,
         1.00000000e+00,  2.00000000e+00,  2.00000000e+00],
       [-1.23308993e+00,  2.31902697e-02,  1.83848970e-03, ...,
         0.00000000e+00,  4.00000000e+00,  3.00000000e+00],
       ...,
       [ 2.22123086e+00, -6.63443368e-01,  4.19175652e-01, ...,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+00],
       [ 1.77551205e+00,  1.39645754e+00, -3.61012523e-02, ...,
         0.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [-7.36319621e-03, -6.63443368e-01, -1.11738390e+00, ...,
         1.00000000e+00,  1.00000000e+00,  3.00000000e+00]])

In [10]:
from sklearn.ensemble import RandomForestClassifier

cls = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 0)),
])
cls.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('trestbps',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='trestbps')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('chol',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='chol')),
                                                                 ('standard',
                                                                  StandardScaler(...
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='slope')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='slope'))])),
                                                ('restecg',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='restecg'))])),
                                                ('ca',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='ca'))])),
                                                ('thal',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='thal'))]))])),
                ('classifier', RandomForestClassifier(random_state=0))])

In [11]:
y_pred=cls.predict(X_test)

Проверим качество

In [12]:
import dill
with open("rf_cls_pipeline.dill", "wb") as f:
    dill.dump(cls, f)